In [17]:
import sys
sys.path.append("../../../repository")
import numpy as np

In [18]:
from src.pulsed_power_ml.models.gupta_model.gupta_clf import GuptaClassifier
from src.pulsed_power_ml.models.gupta_model.gupta_utils import read_power_data_base
from src.pulsed_power_ml.models.gupta_model.gupta_utils import read_parameters

from src.pulsed_power_ml.model_framework.data_io import read_training_files

In [19]:
power_db_file = "/home/thomas/projects/nilm_at_fair/repository/src/pulsed_power_ml/models/gupta_model/apparent_power_data_base.yml"
parameter_file = "/home/thomas/projects/nilm_at_fair/repository/src/pulsed_power_ml/models/gupta_model/parameters.yml"
data_folder = "/home/thomas/projects/nilm_at_fair/training_data/2022-10-25_training_data/tube/"

In [20]:
apparent_power_list = read_power_data_base(power_db_file)
parameter_dict = read_parameters(parameter_file)

In [31]:
clf = GuptaClassifier(
    background_n=parameter_dict["background_n"],
    fft_size_real=parameter_dict["fft_size"] / 2,
    sample_rate=parameter_dict["sample_rate"],
    n_known_appliances=6, # just for this test
    spectrum_type=2,
    n_peaks_max=3,
    apparent_power_list=apparent_power_list,
)

In [32]:
dummy_X = np.random.rand(9, 9)
dummy_y = np.array([[1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0]] * 3)

In [33]:
clf.fit(X=dummy_X, y=dummy_y)

GuptaClassifier(apparent_power_list=[('fluorescent_0', 50.0),
                                     ('halogen_0', 45.9), ('led_0', 5.8)],
                n_known_appliances=6, n_peaks_max=3, spectrum_type=2)

In [34]:
data_array = read_training_files(data_folder,
                                 fft_size=parameter_dict["fft_size"])

In [35]:
state_vector_list = list()
for data_point in data_array:
    state_vector = clf.predict(data_point)
    state_vector_list.append(state_vector)

Event detected: Calculate feature vector


/home/thomas/.venv/nilm_at_fair/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Event detected: Calculate feature vector


/home/thomas/.venv/nilm_at_fair/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [36]:
data_point.shape

(196612,)

In [37]:
state_vector

array([50.,  0.,  0.,  0.,  0.,  0.,  0.])

In [28]:
type(clf)

src.pulsed_power_ml.models.gupta_model.gupta_clf.GuptaClassifier

In [29]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

In [30]:
initial_type = [('float_input', FloatTensorType([None, 196612]))]
onx = convert_sklearn(clf, initial_types=initial_type)

MissingShapeCalculator: Unable to find a shape calculator for type '<class 'src.pulsed_power_ml.models.gupta_model.gupta_clf.GuptaClassifier'>'.
It usually means the pipeline being converted contains a
transformer or a predictor with no corresponding converter
implemented in sklearn-onnx. If the converted is implemented
in another library, you need to register
the converted so that it can be used by sklearn-onnx (function
update_registered_converter). If the model is not yet covered
by sklearn-onnx, you may raise an issue to
https://github.com/onnx/sklearn-onnx/issues
to get the converter implemented or even contribute to the
project. If the model is a custom model, a new converter must
be implemented. Examples can be found in the gallery.
